In [60]:
from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
import math
from tensorflow.python.data import Dataset

In [61]:
import numpy as np

In [62]:
my_df = pd.read_csv("symptoms.csv", sep=",")

In [63]:
my_df.head()

,_id,chief_complaint_id,chief_complaint_name_id,name,reason_code,life_threatening,msp_enabled,primary_complaint,diagnosis_count
0,1,1,1,Upper abdominal pain,15453.0,0.0,1.0,1.0,44
1,2,1,2,Upper belly pain,15453.0,0.0,1.0,0.0,44
2,4,2,4,Lower abdominal pain,15452.0,0.0,1.0,1.0,48
3,5,2,5,Lower belly pain,15452.0,0.0,1.0,0.0,48
4,6,3,6,Abscess (Collection of pus),0.0,0.0,0.0,0.0,17


In [5]:
my_df['life_threatening']=my_df.life_threatening.astype(int)

In [6]:
# my_df['life_threatening'] = my_df['life_threatening'].map({0: "False", 1: 'True'})

In [9]:
modified=my_df.fillna(0)

In [6]:
modified.isnull().sum()

NameError: name 'modified' is not defined

In [64]:
diag_feat = tf.feature_column.numeric_column('diagnosis_count')
name_feat = tf.feature_column.categorical_column_with_hash_bucket('name',hash_bucket_size=390)
# life_feat = tf.feature_column.categorical_column_with_hash_bucket('life_threatening',hash_bucket_size=400)

In [65]:
feat_cols = [diag_feat,name_feat]
my_features = {}
my_features["diagnosis_count"] = my_df[["diagnosis_count"]]
# my_features["life_threatening"] = my_df[["life_threatening"]]
my_features["name"] = my_df[["name"]]

In [66]:
%matplotlib inline

In [67]:
x_data=my_df.drop(my_df.iloc[:,[0,1,2,4,5,6,7]],axis=1)

In [68]:
x_data.head()

,name,diagnosis_count
0,Upper abdominal pain,44
1,Upper belly pain,44
2,Lower abdominal pain,48
3,Lower belly pain,48
4,Abscess (Collection of pus),17


In [69]:
labels=my_df['life_threatening']

In [83]:
from sklearn.model_selection import train_test_split
print(x_data.head())

                          name  diagnosis_count
0         Upper abdominal pain               44
1             Upper belly pain               44
2         Lower abdominal pain               48
3             Lower belly pain               48
4  Abscess (Collection of pus)               17


In [85]:
X_train, X_test, y_train, y_test = train_test_split(x_data,labels,test_size=0.33, random_state=101)
print(X_test)

                                          name  diagnosis_count
38                                Ear pressure               21
188                          Bleeding in brain               11
162                         Behavioral problem               20
100                          Painful urination               18
156                                    Sadness               29
128             Swallowing problem (Dysphagia)               28
161                          Failure to thrive               13
256                               Gum swelling               11
266                               Itching feet               29
385                             Lack of energy               50
291                                Leg redness               37
98                                  Overweight               16
331                             Blood in stool               16
224                                Swollen eye               12
350                                 Skin

In [72]:
input_func=tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=5,num_epochs=1000,shuffle=True)

In [73]:
model=tf.estimator.LinearClassifier(feature_columns=feat_cols,n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\pc\\AppData\\Local\\Temp\\tmprxqc65k5', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000023683FD7860>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [74]:
model.train(input_fn=input_func,steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\pc\AppData\Local\Temp\tmprxqc65k5\model.ckpt.
INFO:tensorflow:loss = 3.465736, step = 1
INFO:tensorflow:global_step/sec: 316.209
INFO:tensorflow:loss = 3.7256544, step = 101 (0.301 sec)
INFO:tensorflow:global_step/sec: 541.271
INFO:tensorflow:loss = 0.40300968, step = 201 (0.200 sec)
INFO:tensorflow:global_step/sec: 542.176
INFO:tensorflow:loss = 0.20003146, step = 301 (0.169 sec)
INFO:tensorflow:global_step/sec: 591.231
INFO:tensorflow:loss = 0.54423743, step = 401 (0.185 sec)
INFO:tensorflow:global_step/sec: 541.514
INFO:tensorflow:loss = 0.30383453, step = 501 (0.169 sec)
INFO:tensorflow:global_step/sec: 499.084
INFO:tensorflow:loss = 1.993311, step = 601 (0.200 sec)
INFO:tensorflow:global_step/s

In [75]:
my_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 389 entries, 0 to 388
Data columns (total 9 columns):
_id                        389 non-null int64
chief_complaint_id         389 non-null int64
chief_complaint_name_id    389 non-null int64
name                       389 non-null object
reason_code                389 non-null float64
life_threatening           389 non-null float64
msp_enabled                389 non-null float64
primary_complaint          389 non-null float64
diagnosis_count            389 non-null int64
dtypes: float64(4), int64(4), object(1)
memory usage: 27.4+ KB


In [76]:
eval_input_func = tf.estimator.inputs.pandas_input_fn(x=X_test,y=y_test,batch_size=10,num_epochs=1,shuffle=False)

In [77]:
results_new = model.evaluate(eval_input_func)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-11-20-09:17:07
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\pc\AppData\Local\Temp\tmprxqc65k5\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-11-20-09:17:08
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.9844961, accuracy_baseline = 0.9844961, auc = 0.78346467, auc_precision_recall = 0.06500008, average_loss = 0.08332825, global_step = 1000, label/mean = 0.015503876, loss = 0.8268726, precision = 0.0, prediction/mean = 0.037178893, recall = 0.0
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: C:\Users\pc\AppData\Local\Temp\tmprxqc65k5\model.ckpt-1000


In [78]:
results_new

{'accuracy': 0.9844961,
 'accuracy_baseline': 0.9844961,
 'auc': 0.78346467,
 'auc_precision_recall': 0.06500008,
 'average_loss': 0.08332825,
 'label/mean': 0.015503876,
 'loss': 0.8268726,
 'precision': 0.0,
 'prediction/mean': 0.037178893,
 'recall': 0.0,
 'global_step': 1000}

In [79]:
print(X_train.head())

                           name  diagnosis_count
124  Aphasia (Trouble speaking)               34
216                   Sore skin               19
60                    Head pain               71
133                      Trauma                9
141                Vaginal pain               11


In [92]:
Xnew = np.array([[9, 44]])

In [93]:
inp=tf.estimator.inputs.numpy_input_fn({'x':Xnew},shuffle=False)

In [94]:
list(model.predict(input_fn=inp))

INFO:tensorflow:Calling model_fn.


ValueError: Feature diagnosis_count is not in features dictionary.